In [1]:
from Bio import Entrez
import eutils

In [2]:
Entrez.email = 'your.email@example.com'

#1. (esearch) ищет PubMed статьи по интересному для вас запросу

In [3]:
handle = Entrez.esearch(db = "pubmed", term = "crustacyanin")
record = Entrez.read(handle)
print(record)

{'Count': '65', 'RetMax': '20', 'RetStart': '0', 'IdList': ['35010161', '34436301', '33919403', '33465290', '32851672', '32596057', '32236233', '31813041', '30860355', '29683674', '29178679', '28851818', '26220698', '25797168', '25605312', '24782450', '23570752', '23510436', '23441225', '22869108'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'crustacyanin[All Fields]', 'Field': 'All Fields', 'Count': '65', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'crustacyanin[All Fields]'}


#2. (efetch) возвращает асбтракты от первых трех статей в простом текстовом файле

In [4]:
mshandle = Entrez.efetch(db="pubmed", id=record["IdList"][0:2], rettype="abstract", retmode="text")
mshandle.read()

'\n1. Foods. 2021 Dec 23;11(1). pii: 35. doi: 10.3390/foods11010035.\n\nPurification and Characterisation of Two Novel Pigment Proteins from the Carapace\nof Red Swamp Crayfish (Procambarus clarkii).\n\nChen H(1)(2), Ji H(1)(3)(4)(5)(6), Pan C(6)(7), Zhang D(1)(3)(4)(5), Su\nW(1)(3)(4)(5), Liu S(1)(3)(4)(5)(6), Deng Y(1), Huang X(1).\n\nAuthor information: \n(1)Guangdong Provincial Key Laboratory of Aquatic Product Processing and Safety, \nCollege of Food Science and Technology, Guangdong Ocean University, Zhanjiang\n524088, China.\n(2)Hunan Provincial Key Laboratory of Soybean Products Processing and Safety\nControl, College of Food and Chemical Engineering, Shaoyang University, Shaoyang \n422000, China.\n(3)Guangdong Provincial Engineering Technology Research Center of Seafood,\nCollege of Food Science and Technology, Guangdong Ocean University, Zhanjiang\n524088, China.\n(4)Guangdong Province Engineering Laboratory for Marine Biological Products,\nCollege of Food Science and Technol

#3. запрашивает в базе нуклеотидных последовательностей все последовательности по названию гена для организма по названию вида и возвращает xml

In [13]:
handle = Entrez.esearch(db = "nucleotide", term = "agrotis AND Agrotis exclamationis[orgn]")
record = Entrez.read(handle)
Entrez.efetch(db = "nucleotide", id = record["IdList"])

<_io.TextIOWrapper encoding='UTF-8'>

#4. ищет ID организма по названию в базе

In [14]:
handle = Entrez.esearch(db = "taxonomy", term = "Agrotis exclamationis")
record = Entrez.read(handle)
print(record)
print(record['IdList'])

{'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': ['215162'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'Agrotis exclamationis[All Names]', 'Field': 'All Names', 'Count': '1', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'Agrotis exclamationis[All Names]'}
['215162']


#5. запрашивает в базе нуклеотидных последовательностей по названию гена,  после чего возвращает таблицу с UID (в XML это поле называется Id), accession number (в XML это поле называется Caption), длиной последовательности (Slen)

In [15]:
handle = Entrez.esearch(db="protein", term="agrotis AND Agrotis exclamationis[orgn]")
record = Entrez.read(handle)
with open("id_capt_len_py.txt", "w") as ouf:
    for rec in record["IdList"]:
        temphandle = Entrez.read(Entrez.esummary(db="protein", id=rec, retmode="text"))
        ouf.write(temphandle[0]['Id']+"\t"+temphandle[0]['Caption']+"\t"+str(int(temphandle[0]['Length']))+"\n")

#6. даёт в базу нуклеотидных или белковых последовательностей текстовый, а затем возвращает  последовательности в  формате fasta,  которые записывает в файл

In [20]:
handle = Entrez.esearch(db="protein", term="agrotis AND Agrotis exclamationis[orgn]")
record = Entrez.read(handle)
Entrez.efetch(db="protein", id=record["IdList"], retmode="text", rettype="fasta").read()
with open("agex.fasta", "w") as ouf:
    for rec in record["IdList"]:
        lne = Entrez.efetch(db="protein", id=rec, retmode="text", rettype="fasta").read()
        ouf.write(lne+"\n")

#7. скачивает белок, соответствующий известному UID нуклеотида

In [18]:
lhandle = Entrez.elink(dbfrom="nucleotide", db="protein", id="2065188392") ##pubmed default
lrecord = Entrez.read(lhandle)
prothandle = lrecord[0]["LinkSetDb"][0]['Link'][0]['Id']
rrecord = Entrez.efetch(db="protein", id=prothandle, rettype="fasta", retmode="text")
with open ("prot_from_nt.fasta", "w") as ouf:
    ouf.write(rrecord.read()+"\n")

#8. скачивает все последовательности из работы с PMID  … (например, из первого задания) и пишет их в файл fasta

In [19]:
lhandle = Entrez.elink(dbfrom="pubmed", db="nucleotide", id="20558169")
lrecord = Entrez.read(lhandle)
ids = []
for el in lrecord[0]["LinkSetDb"][0]["Link"]:
    ids.append(el['Id'])
rrecord = Entrez.efetch(db="nucleotide", id=ids[:4], rettype="fasta", retmode="text")
with open ("py_fasta_pmid.fasta", "w") as ouf:
    ouf.write(rrecord.read()+"\n")